### Minibatch placeholders

In [51]:
import tensorflow as tf

d = 120
x = tf.placeholder(tf.float32, (None, d))
y = tf.placeholder(tf.float32, (None))

### Hidden Layer

In [55]:
n_hidden = 120
w = tf.Variable(tf.random_normal((d, n_hidden)))
b = tf.Variable(tf.random_normal((d,)))
x_hidden = tf.nn.relu(tf.matmul(x, w) + b)
#Adding Dropout
keep_prob = tf.placeholder(tf.float32)
x_hidden = tf.nn.dropout(x_hidden, keep_prob)

### Output Unit

In [56]:
w_out = tf.Variable(tf.random_normal((d, 1)))
b_out = tf.Variable(tf.random_normal((1,)))
y_logit = tf.matmul(x_hidden, w_out) + b_out
y_pred = tf.round(tf.sigmoid(y_logit))

### Cross-Entropy Loss

In [57]:
y_expand = tf.expand_dims(y, 1)
entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_logit, labels=y_expand)
l = tf.reduce_sum(entropy)

### Optimizer

In [58]:
train_op = tf.train.AdamOptimizer(.01).minimize(l)

### Getting IRIS dataset

In [59]:
import os
import requests

import tensorflow as tf
import numpy as np

IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

if not os.path.exists(IRIS_TRAINING):
    raw = requests.get(IRIS_TRAINING_URL).content
    with open(IRIS_TRAINING,'wb') as f:
        f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = requests.get(IRIS_TEST_URL).content
    with open(IRIS_TEST,'wb') as f:
        f.write(raw)
        
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=IRIS_TRAINING,target_dtype=np.int,features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=IRIS_TEST, target_dtype=np.int,features_dtype=np.float32)

### Training

In [64]:
# import numpy as np
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for step in range(10000):
#         feed_dict = {x: np.transpose(training_set.data), y: np.transpose(training_set.target), keep_prob: 0.5}
#         _, loss = sess.run([train_op, l], feed_dict=feed_dict)          

InvalidArgumentError: Incompatible shapes: [4,1] vs. [120,1]
	 [[Node: logistic_loss_4/mul = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](add_14, ExpandDims_4)]]

Caused by op 'logistic_loss_4/mul', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/sohaibfarooqi/projects/tensorflow-for-deep-learning/env/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/sohaibfarooqi/projects/tensorflow-for-deep-learning/env/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/sohaibfarooqi/projects/tensorflow-for-deep-learning/env/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, i